[![preview notebook](https://img.shields.io/static/v1?label=render%20on&logo=github&color=87ce3e&message=GitHub)](https://github.com/open-atmos/PySDM/blob/main/examples/PySDM_examples/Bolin_1958/table_1.ipynb)
[![launch on mybinder.org](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/open-atmos/PySDM.git/main?urlpath=lab/tree/examples/PySDM_examples/Bolin_1958/table_1.ipynb)
[![launch on Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/open-atmos/PySDM/blob/main/examples/PySDM_examples/Bolin_1958/table_1.ipynb)

### based on Table 1 from B.Bolin 1958 "On the use of tritium as a tracer for water in nature" (https://digitallibrary.un.org/record/3892725)

In [1]:
import sys
if 'google.colab' in sys.modules:
    !pip --quiet install open-atmos-jupyter-utils
    from open_atmos_jupyter_utils import pip_install_on_colab
    pip_install_on_colab('PySDM-examples')

In [8]:
import numpy as np
import pandas
from PySDM.physics import in_unit, si
from PySDM import Formulae
from IPython.display import display

In [3]:
radii = np.asarray([0.005, 0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.20]) * si.cm

In [9]:
formulae = Formulae(
    terminal_velocity='RogersYau',
    drop_growth='Mason1951',
    diffusion_thermics='Neglect',
    saturation_vapour_pressure='AugustRocheMagnus',
    ventilation='Froessling1938',
    particle_shape_and_density='LiquidSpheres',
    air_dynamic_viscosity='ZografosEtAl1987',
    constants={'BOLIN_ISOTOPE_TIMESCALE_COEFF_C1': 1.63},
    isotope_relaxation_timescale='Bolin1958',
)
const = formulae.constants
any_non_zero_value = 44.
temperature = const.T0 + 10 * si.K
pressure = const.p_STP
pvs = formulae.saturation_vapour_pressure.pvs_water(temperature)
v_term = formulae.terminal_velocity.v_term(radii)
eta_air=formulae.air_dynamic_viscosity.eta_air(temperature)
D=formulae.diffusion_thermics.D(T=temperature, p=pressure)

air_density = pressure/const.Rd/temperature
assert abs(air_density - 1)/air_density <.3
Re = formulae.particle_shape_and_density.reynolds_number(
     radius=radii,
     velocity_wrt_air=v_term,
     dynamic_viscosity=eta_air,
     density=air_density,
)
Sc = formulae.trivia.air_schmidt_number(
    dynamic_viscosity=eta_air, 
    diffusivity=D, 
    density=air_density,
)
F = formulae.ventilation.ventilation_coefficient(sqrt_re_times_cbrt_sc=Re**(1/2) * Sc**(1/3))

r_dr_dt = formulae.drop_growth.r_dr_dt(
    RH_eq=1,
    T=temperature,
    RH=0,
    lv=0,
    pvs=pvs,
    D=D,
    K=any_non_zero_value,
    ventilation_factor=F
)
adjustment_time = formulae.isotope_relaxation_timescale.tau(radius = radii, r_dr_dt = r_dr_dt)


pandas.options.display.float_format = '{:>,.2g}'.format
data = pandas.DataFrame({
    'radius [cm]': in_unit(radii, si.cm),
    'adjustment time [s]': adjustment_time,
    'terminal velocity [m/s]': v_term,
    'distance [m]': v_term * adjustment_time,
})

display(data)

,radius [cm],adjustment time [s],terminal velocity [m/s],distance [m]
0,0.005,1.7,0.4,0.69
1,0.01,5.4,0.8,4.3
2,0.025,20,2,40
3,0.05,48,4,1.9e+02
4,0.075,81,5.5,4.4e+02
5,0.1,1.2e+02,6.4,7.6e+02
6,0.15,2e+02,7.8,1.6e+03
7,0.2,3e+02,9,2.7e+03
